In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import timedelta

Read CDS data

In [2]:
cds_6m = pd.read_excel('cds.xlsx', sheet_name='6mth').dropna(axis= 0, how='all').add_suffix('_6m')[['Date_6m', 'Open_6m']]
cds_1yr = pd.read_excel('cds.xlsx', sheet_name='1yr').dropna(axis= 0, how='all').add_suffix('_1yr')[['Date_1yr', 'Open_1yr']]
cds_2yr  = pd.read_excel('cds.xlsx', sheet_name='2yr').dropna(axis= 0, how='all').add_suffix('_2yr')[['Date_2yr', 'Open_2yr']]
cds_3yr = pd.read_excel('cds.xlsx', sheet_name='3yr').dropna(axis= 0, how='all').add_suffix('_3yr')[['Date_3yr', 'Open_3yr']]
cds_5yr = pd.read_excel('cds.xlsx', sheet_name='5yr').dropna(axis= 0, how='all').add_suffix('_5yr')[['Date_5yr', 'Open_5yr']]
cds_7yr  = pd.read_excel('cds.xlsx', sheet_name='7yr').dropna(axis= 0, how='all').add_suffix('_7yr')[['Date_7yr', 'Open_7yr']]
cds_10yr  = pd.read_excel('cds.xlsx', sheet_name='10yr').dropna(axis= 0, how='all').add_suffix('_10yr')[['Date_10yr', 'Open_10yr']]
cds_15yr = pd.read_excel('cds.xlsx', sheet_name='15yr').dropna(axis= 0, how='all').add_suffix('_15yr')[['Date_15yr', 'Open_15yr']]
cds_20yr  = pd.read_excel('cds.xlsx', sheet_name='20yr').dropna(axis= 0, how='all').add_suffix('_20yr')[['Date_20yr', 'Open_20yr']]
cds_30yr  = pd.read_excel('cds.xlsx', sheet_name='30yr').dropna(axis= 0, how='all').add_suffix('_30yr')[['Date_30yr', 'Open_30yr']]

In [3]:
cds = cds_6m.merge(cds_1yr, left_on='Date_6m', right_on= 'Date_1yr', how='inner', copy = False).merge(cds_2yr, left_on='Date_6m', right_on= 'Date_2yr', how='inner', copy = False).merge(cds_3yr, left_on='Date_6m', right_on= 'Date_3yr', how='inner', copy = False).merge(cds_5yr, left_on='Date_6m', right_on= 'Date_5yr', how='inner', copy = False).merge(cds_7yr, left_on='Date_6m', right_on= 'Date_7yr', how='inner', copy = False).merge(cds_10yr, left_on='Date_6m', right_on= 'Date_10yr', how='inner', copy = False)

In [4]:
cds = cds[cds['Date_6m'] < '2022-04-01']
cds = cds.drop(columns=['Date_1yr', 'Date_2yr', 'Date_3yr', 'Date_5yr', 'Date_7yr', 'Date_10yr'])
cds = cds.rename(columns={'Date_6m':'Settlement Date'})

In [5]:
cds['Maturity_6m'] = cds['Settlement Date'] + pd.DateOffset(days = 180)
cds['Maturity_1yr'] = cds['Settlement Date'] + pd.DateOffset(years =1)
cds['Maturity_2yr'] = cds['Settlement Date'] + pd.DateOffset(years =2)
cds['Maturity_3yr'] = cds['Settlement Date'] + pd.DateOffset(years =3)
cds['Maturity_5yr'] = cds['Settlement Date'] + pd.DateOffset(years =5)
cds['Maturity_7yr'] = cds['Settlement Date'] + pd.DateOffset(years =7)
cds['Maturity_10yr'] = cds['Settlement Date'] + pd.DateOffset(years =10)
cds = cds.reset_index(drop=True)

In [6]:
cds.head()

,Settlement Date,Open_6m,Open_1yr,Open_2yr,Open_3yr,Open_5yr,Open_7yr,Open_10yr,Maturity_6m,Maturity_1yr,Maturity_2yr,Maturity_3yr,Maturity_5yr,Maturity_7yr,Maturity_10yr
0,2022-03-31,7.740,8.905,10.925,12.170,15.875,18.535,20.825,2022-09-27,2023-03-31,2024-03-31,2025-03-31,2027-03-31,2029-03-31,2032-03-31
1,2022-03-30,7.880,8.990,10.950,12.460,16.460,19.420,21.670,2022-09-26,2023-03-30,2024-03-30,2025-03-30,2027-03-30,2029-03-30,2032-03-30
2,2022-03-29,7.910,9.035,10.985,12.485,16.460,19.420,21.660,2022-09-25,2023-03-29,2024-03-29,2025-03-29,2027-03-29,2029-03-29,2032-03-29
3,2022-03-28,7.880,8.990,10.930,12.425,16.400,19.300,21.910,2022-09-24,2023-03-28,2024-03-28,2025-03-28,2027-03-28,2029-03-28,2032-03-28
4,2022-03-25,7.855,8.970,10.920,12.420,16.385,19.350,21.580,2022-09-21,2023-03-25,2024-03-25,2025-03-25,2027-03-25,2029-03-25,2032-03-25


In [7]:
cds = cds [['Settlement Date', 'Maturity_6m', 'Maturity_1yr',
       'Maturity_2yr', 'Maturity_3yr', 'Maturity_5yr', 'Maturity_7yr',
       'Maturity_10yr', 'Open_6m', 'Open_1yr', 'Open_2yr', 'Open_3yr',
       'Open_5yr', 'Open_7yr', 'Open_10yr']]

## Discount process

In [8]:
discount = pd.read_csv('discount_factors.csv')
discount = discount.rename(columns={'dates':'Date'})
discount['Date'] = pd.to_datetime(discount['Date'])
discount = discount.set_index('Date')

In [9]:
rate = discount.applymap(lambda x : -np.log(x))

In [10]:
mths = np.array([1/12, 3/12, 1/2, 9/12])
yrs = np.arange(1, 31, 1)
time_periods = np.append(mths, yrs)

In [11]:
for i, val in enumerate(time_periods):
    rate.iloc[:, i] = rate.iloc[:, i]/val
rate = rate.reset_index()

In [12]:
rate['Maturity_1m'] = rate['Date'] + pd.DateOffset(months = 1)
rate['Maturity_3m'] = rate['Date'] + pd.DateOffset(months = 3)
rate['Maturity_6m'] = rate['Date'] + pd.DateOffset(months = 6)
rate['Maturity_9m'] = rate['Date'] + pd.DateOffset(months = 9)
rate['Maturity_1yr'] = rate['Date'] + pd.DateOffset(years =1)
rate['Maturity_2yr'] = rate['Date'] + pd.DateOffset(years =2)
rate['Maturity_3yr'] = rate['Date'] + pd.DateOffset(years =3)
rate['Maturity_4yr'] = rate['Date'] + pd.DateOffset(years =4)
rate['Maturity_5yr'] = rate['Date'] + pd.DateOffset(years =5)
rate['Maturity_6yr'] = rate['Date'] + pd.DateOffset(years =6)
rate['Maturity_7yr'] = rate['Date'] + pd.DateOffset(years =7)
rate['Maturity_8yr'] = rate['Date'] + pd.DateOffset(years =8)
rate['Maturity_9yr'] = rate['Date'] + pd.DateOffset(years =9)
rate['Maturity_10yr'] = rate['Date'] + pd.DateOffset(years =10)
rate['Maturity_11yr'] = rate['Date'] + pd.DateOffset(years =11)

In [13]:
rate = rate[['Date', 'Maturity_1m', 'Maturity_3m', 'Maturity_6m',
       'Maturity_9m', 'Maturity_1yr', 'Maturity_2yr', 'Maturity_3yr',
       'Maturity_4yr', 'Maturity_5yr', 'Maturity_6yr', 'Maturity_7yr',
       'Maturity_8yr', 'Maturity_9yr', 'Maturity_10yr', 'Maturity_11yr', '1m', '3m', '6m', '9m', '12m', '2y', '3y', '4y', '5y', '6y',
       '7y', '8y', '9y', '10y', '11y' ]]

In [14]:
common_dates = [x for x in rate.Date if x in list(cds['Settlement Date'])]

In [15]:
rate = rate[rate['Date'].isin(common_dates)]
cds = cds[cds['Settlement Date'].isin(common_dates)]

### Write files to csv

In [16]:
rate.to_csv('interest_rates_from_df.csv', index=False)
cds.to_csv("./cds_data_processed.csv", index=False)